### **Purpose: ** 

In [1]:
player = 'Darren Appleton'
team = 'e'

In [2]:
import pickle
import pandas as pd

In [5]:
path = '//DREW/Users/andrew/Desktop/mosconi/'
dframe = pickle.load(open(path+'pkl/allyears_clean_locs','rb'))
if 'a' in team.lower():
    df = dframe[dframe.loc[:,'American_player'].str.contains(player)]
    df['Europe_lost'] = ~ df.loc[:,'Europe_won']
    df = df.loc[:,['Format', 'European_player', 'European_score', 'American_score',
       'American_player', 'Europe_lost','Europe_won']]
    df.columns = ['Format', 'European_player', 'European_score', 'American_score',
       'American_player', 'America_won', 'America_lost']
else:
    df = dframe[dframe.loc[:,'European_player'].str.contains(player)]
    df['Europe_lost'] = ~ df.loc[:,'Europe_won']

c:\users\andrew\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def stats(df):
    try:
        if 'a' in team.lower():
            win = df['America_won'].sum()
            loss = df['America_lost'].sum()
        else:
            win = df['Europe_won'].sum()
            loss = df['Europe_lost'].sum()
        mp= win+loss
        wl = 'Win-loss: {}-{}'.format(win,loss)
        pct = round(win/(win+loss)*100,0)
        return('Matches Played: {}'.format(mp),wl,'Pct: {}'.format(pct))
    except:
        return('No matches to display')

### Lets have a quick look at his overall stats by checking out his singles, doubles, and team stats.

In [ ]:
sing = df[df['Format']=='Singles']
dub = df[df['Format']=='Doubles']
teams = df[df['Format']=='Teams']
triples = df[df['Format']=='Triples'] 

In [ ]:
'Overall: {}'.format(stats(df))
'Singles: {}'.format(stats(sing))
'Doubles: {}'.format(stats(dub))
'Teams:   {}'.format(stats(teams))
'Triples: {}'.format(stats(triples))

## So he plays *significantly* better in singles matches (71% doubles vs 40% singles)!!!

### Let's see if he plays better with certain partners. 

In [ ]:
if 'a' in team.lower():
    dub = dub[['European_player','American_player','America_won','America_lost']]
    dub['Partner']=dub['American_player'].str.replace(player,'')
    dub['Partner']=dub['Partner'].str.replace('&','')
    dub['Partner']=dub['Partner'].str.strip()
    dub.groupby('Partner').sum().sort_values(['America_won','America_lost'],ascending=False)    
else:
    dub = dub[['European_player','American_player','Europe_won','Europe_lost']]
    dub['Partner']=dub['European_player'].str.replace(player,'')
    dub['Partner']=dub['Partner'].str.replace('&','')
    dub['Partner']=dub['Partner'].str.strip()
    dub.groupby('Partner').sum().sort_values(['Europe_won','Europe_lost'],ascending=False)

### Looks like he plays best when partnered with Boyes or Souquet and plays bad with most other players.

More questions to answer:
1. Does he play better or worse against certain opponents?
2. Has his performance changed over the years?
3. Is performance affected by location (home vs away)?

## Let's see if he performs better or worse against certain opponents in singles matches.

In [ ]:
if 'a' in team.lower():
    sing.groupby('European_player').sum().sort_values('America_won',ascending = False)
else:
    sing.groupby('American_player').sum().sort_values('Europe_won',ascending = False)


## Apparently, he does *really* well against Shane (6-1)!!!
## He also doer somewhat poorly against Archer. 
## Let's see what his record was for each year.

In [ ]:
lst =df.index.values.tolist() 
years = set([])
for item in lst:
    years.add(item[0])
years=list(years)

In [ ]:
yearly=[]
for year in years:
    if 'a' in team:
        win = df.loc[year]['America_won'].sum()
        loss = df.loc[year]['America_lost'].sum()
    else:
        win = df.loc[year]['Europe_won'].sum()
        loss = df.loc[year]['Europe_lost'].sum()
    yearly.append((win,loss))

In [ ]:
yr= {}
for k,v in zip(years,yearly):
    yr[k]=v

In [ ]:
yr

In [ ]:
win=[]
loss=[]
pct=[]
for w,l in yearly:
    win.append(w)
    loss.append(l)
    p=round((w*100/(w+l)),0)
    pct.append(p)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ind = np.arange(len(years))  # the x locations for the groups

fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)

ax.plot(pct,linestyle='-',marker='.',markersize=12)
ax.axhline(50, color='r',linestyle='--')

# add some;
ax.set_ylabel('Percentage of Games Won');
ax.set_title('Yearly Performance Average');
ax.set_xticks(ind);
ax.set_xticklabels( (years),rotation=30 );
# ax.legend( (rects1[0], rects2[0]), ('Wins', 'Losses') );

### So his worst years were: 2009 and his best years were: 2010,2013. Let's dig a bit deeper to see how big of a factor location is for his performance.
### Let's check how the team did each year.

In [ ]:
def wincount(dframe=dframe,years=years):
    l=0
    w=0
    tie=0
    for year in years:
        f,t = dframe.loc[year]['Europe_won'].value_counts(sort=False)
        print(f,t)
        if f>t:
            l+=1
        elif f<t:
            w+=1
        else:
            tie+=1
    return 'win-loss-tie: {}-{}-{}'.format(w,l,tie)
wincount()

### Europe went 7-1 with him on the team

In [ ]:
locs = pickle.load(open(path+'pkl/dloc','rb'))

In [ ]:
hm = []
aw = []

for year in years:
    if 'USA' in locs[year]:
        if 'a' in team.lower():
            hm.append(year)
        else:
            aw.append(year)
    else:
        if 'a' in team.lower():
            aw.append(year)
        else:
            hm.append(year)
away = df.loc[aw]
home = df.loc[hm]

In [ ]:
'Played home {} times, away {} times'.format(len(hm),len(aw))

In [ ]:
'Home Games: {}'.format(stats(home))

In [ ]:
'Away Games: {}'.format(stats(away))

### So he plays better at home.

### Let's drill down a bit further to parse out his singles and doubles matches for both home and away 

In [ ]:
hsing = stats(home[home['Format']=='Singles'])

In [ ]:
hdub = stats(home[home['Format']=='Doubles'])

In [ ]:
asing = stats(away[away['Format']=='Singles'])

In [ ]:
adub = stats(away[away['Format']=='Doubles'])

In [ ]:
'Home Singles: {}'.format(hsing)
'Home Doubles: {}'.format(hdub)
'Away Singles: {}'.format(asing)
'Away Doubles: {}'.format(adub)

## Excellent singles record at home! (7-1)

# Summary

- Darren has performed *far* better in singles than doubles (71% vs. 40%)! 
- He has played *exceptionally* well against Shane Van Boening (6-1) 
- His singles record at home is *superb* (7-1)
- His play has been relatively consistent over the years
- He is **weak** in doubles matches, unless partnered with Boyes or Souquet

- He is somewhat **weak** in singles against Johnny Archer (0-2)
- Europe has a strong record with him on the team (7-1)